In [1]:
using BenchmarkTools

In [2]:
import Base.LinAlg.sylvester

sylvester(a::Union{Real,Complex},b::Union{Real,Complex},c::Union{Real,Complex}) = -c / (a + b)

function _sqrtm(A::UpperTriangular)
    realmatrix = false
    if isreal(A)
        realmatrix = true
        for i = 1:Base.LinAlg.checksquare(A)
            if real(A[i,i]) < 0
                realmatrix = false
                break
            end
        end
    end
    _sqrtm(A,Val{realmatrix})
end
function _sqrtm{T,realmatrix}(A::UpperTriangular{T},::Type{Val{realmatrix}})
    B = A.data
    n = Base.LinAlg.checksquare(B)
    t = realmatrix ? typeof(sqrt(zero(T))) : typeof(sqrt(complex(zero(T))))
    R = zeros(t, n, n)
    tt = typeof(zero(t)*zero(t))
    @inbounds for j = 1:n
        R[j,j] = realmatrix ? sqrt(B[j,j]) : sqrt(complex(B[j,j]))
        for i = j-1:-1:1
            r::tt = B[i,j]
            @simd for k = i+1:j-1
                r -= R[i,k]*R[k,j]
            end
            r==0 || (R[i,j] = sylvester(R[i,i],R[j,j],-r))
        end
    end
    return UpperTriangular(R)
end

_sqrtm (generic function with 2 methods)

In [3]:
A = randn(127,127)
A = UpperTriangular(schurfact(A'*A)[:T]);

In [4]:
@benchmark sqrtm(A)

BenchmarkTools.Trial: 
  memory estimate:  21.08 mb
  allocs estimate:  1373642
  --------------
  minimum time:     22.486 ms (0.00% GC)
  median time:      23.981 ms (5.33% GC)
  mean time:        24.644 ms (5.04% GC)
  maximum time:     30.715 ms (0.00% GC)
  --------------
  samples:          203
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [5]:
@benchmark _sqrtm(A)

BenchmarkTools.Trial: 
  memory estimate:  126.16 kb
  allocs estimate:  3
  --------------
  minimum time:     684.498 μs (0.00% GC)
  median time:      704.194 μs (0.00% GC)
  mean time:        723.343 μs (0.62% GC)
  maximum time:     1.860 ms (49.43% GC)
  --------------
  samples:          6889
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%